In [ ]:
import os
import cv2

main_directory = 'C:/Users/User/Desktop/Fangorn/data/final'
main_directory_2 = 'C:/Users/User/Desktop/Helevorn/data'
sets = ['images']
types = ['test', 'train', 'val']

# Define color spaces and their OpenCV conversions
color_spaces = {
    'RGB': (None, ['RGB_B', 'RGB_G', 'RGB_R']),  # No conversion needed
    'GRAY': (cv2.COLOR_BGR2GRAY, ['gray']),
    'YUV': (cv2.COLOR_BGR2YUV, ['YUV_Y', 'YUV_U', 'YUV_V']),
    'HSV': (cv2.COLOR_BGR2HSV, ['HSV_H', 'HSV_S', 'HSV_V']),
    'HLS': (cv2.COLOR_BGR2HLS, ['HLS_H', 'HLS_L', 'HLS_S']),
    'CIELab': (cv2.COLOR_BGR2Lab, ['CIELab_L', 'CIELab_a', 'CIELab_b']),
    'YCrCb': (cv2.COLOR_BGR2YCrCb, ['YCrCb_Y', 'YCrCb_Cr', 'YCrCb_Cb']),
}

# Function to create necessary directories
def create_output_dirs():
    for space, (_, channels) in color_spaces.items():
        for c in channels:
            for s in sets:
                for t in types:
                    path = os.path.join(main_directory_2, 'chan', c, s, t)
                    os.makedirs(path, exist_ok=True)

# Apply CLAHE and save channel images
def process_images():
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    for s in sets:
        for t in types:
            src_dir = os.path.join(main_directory, s, t)
            for filename in os.listdir(src_dir):
                fpath = os.path.join(src_dir, filename)
                if not os.path.isfile(fpath):
                    continue
                
                image = cv2.imread(fpath)

                for space, (conversion, channels) in color_spaces.items():
                    if space == 'GRAY':
                        gray = cv2.cvtColor(image, conversion)
                        enhanced = clahe.apply(gray)
                        out_path = os.path.join(main_directory_2, 'chan', channels[0], s, t, filename)
                        cv2.imwrite(out_path, enhanced)

                    elif space == 'RGB':
                        split_channels = cv2.split(image)
                        for idx, c in enumerate(channels):
                            enhanced = clahe.apply(split_channels[idx])
                            out_path = os.path.join(main_directory_2, 'chan', c, s, t, filename)
                            cv2.imwrite(out_path, enhanced)

                    else:
                        converted = cv2.cvtColor(image, conversion)
                        split_channels = cv2.split(converted)
                        for idx, c in enumerate(channels):
                            enhanced = clahe.apply(split_channels[idx])
                            out_path = os.path.join(main_directory_2, 'chan', c, s, t, filename)
                            cv2.imwrite(out_path, enhanced)

# Run the script
create_output_dirs()
process_images()